In [1]:
!pip install dagshub mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully unin

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
import dagshub
dagshub.init(repo_owner='zeliz22', repo_name='ML_House-Pricing', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=72c228da-a9e6-4f23-9874-08e1f03054fe&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=8e017d20b13705ffdd59eac829b816f0b52528672242d1e61aed36c89dd79433




Accessing as zeliz22

Initialized MLflow to track repo "zeliz22/ML_House-Pricing"

Repository zeliz22/ML_House-Pricing initialized!

In [4]:
train_transaction =  pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

In [5]:
from sklearn.model_selection import train_test_split

def prepare_data(df, target='isFraud', test_size=0.15, random_state=42):
    # Split train/val/test
    train_val, test = train_test_split(
        df, test_size=test_size, stratify=df[target], random_state=random_state
    )
    train, val = train_test_split(
        train_val, 
        test_size=test_size/(1-test_size),  # Adjust for nested split
        stratify=train_val[target],
        random_state=random_state
    )
    
    # Separate X/y
    def _split(df):
        return df.drop(columns=[target, 'TransactionID']), df[target]
    
    X_train, y_train = _split(train)
    X_val, y_val = _split(val)
    X_test, y_test = _split(test)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(train_transaction)

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
from typing import Dict, Optional, Union

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 numeric_strategy: str = 'mean',
                 categorical_strategy: str = 'most_frequent',
                 numeric_fill_value: Optional[Union[int, float]] = None,
                 categorical_fill_value: Optional[str] = None,
                 drop_threshold: float = 0.8):

        self.numeric_strategy = numeric_strategy
        self.categorical_strategy = categorical_strategy
        self.numeric_fill_value = numeric_fill_value
        self.categorical_fill_value = categorical_fill_value
        self.drop_threshold = drop_threshold
        self.numeric_impute_values_ = {}
        self.categorical_impute_values_ = {}
        self.columns_to_drop_ = []

    def fit(self, X: pd.DataFrame, y=None):
        """Learn imputation values from the data"""
        
        # Identify columns to drop
        null_ratios = X.isnull().mean()
        self.columns_to_drop_ = list(null_ratios[null_ratios > self.drop_threshold].index)
        X_clean = X.drop(columns=self.columns_to_drop_)
        
        # Separate numeric and categorical columns
        numeric_cols = X_clean.select_dtypes(include=np.number).columns
        categorical_cols = X_clean.select_dtypes(exclude=np.number).columns
        
        # Calculate numeric imputation values
        for col in numeric_cols:
            if self.numeric_strategy == 'mean':
                self.numeric_impute_values_[col] = X_clean[col].mean()
            elif self.numeric_strategy == 'median':
                self.numeric_impute_values_[col] = X_clean[col].median()
            elif self.numeric_strategy == 'constant':
                if self.numeric_fill_value is None:
                    raise ValueError("numeric_fill_value must be specified for constant strategy")
                self.numeric_impute_values_[col] = self.numeric_fill_value
            elif self.numeric_strategy != 'drop':
                raise ValueError(f"Unknown numeric strategy: {self.numeric_strategy}")
        
        # Calculate categorical imputation values
        for col in categorical_cols:
            if self.categorical_strategy == 'most_frequent':
                self.categorical_impute_values_[col] = X_clean[col].mode()[0]
            elif self.categorical_strategy == 'constant':
                if self.categorical_fill_value is None:
                    raise ValueError("categorical_fill_value must be specified for constant strategy")
                self.categorical_impute_values_[col] = self.categorical_fill_value
            elif self.categorical_strategy != 'drop':
                raise ValueError(f"Unknown categorical strategy: {self.categorical_strategy}")
        
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Apply the learned imputation to new data"""
        
        # Drop high-null columns
        X_clean = X.drop(columns=self.columns_to_drop_)
        
        # Separate numeric and categorical columns
        numeric_cols = X_clean.select_dtypes(include=np.number).columns
        categorical_cols = X_clean.select_dtypes(exclude=np.number).columns
        
        # Apply numeric imputation
        for col in numeric_cols:
            if col in self.numeric_impute_values_:
                X_clean[col] = X_clean[col].fillna(self.numeric_impute_values_[col])
            elif self.numeric_strategy == 'drop':
                X_clean = X_clean.dropna(subset=[col])
        
        # Apply categorical imputation
        for col in categorical_cols:
            if col in self.categorical_impute_values_:
                X_clean[col] = X_clean[col].fillna(self.categorical_impute_values_[col])
            elif self.categorical_strategy == 'drop':
                X_clean = X_clean.dropna(subset=[col])
        
        return X_clean

    def fit_transform(self, X: pd.DataFrame, y=None) -> pd.DataFrame:
        """Fit and transform in one step"""
        return self.fit(X, y).transform(X)

In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

class CustomEncoder:
    def __init__(self, threshold = 3):
        self.threshold = threshold
        
        # Initialize encoders
        self.one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        
        # Store feature names for one-hot encoding
        self.one_hot_feature_names = None
        
    def fit(self, X, y=None):

        cat_cols = [col for col in X.columns if X[col].dtype == 'object']
        s = X[cat_cols].nunique()

        self.ordinal_cols = list(s[s > self.threshold].index)
        self.one_hot_cols = list(s[s <= self.threshold].index)

        if self.one_hot_cols:
            self.one_hot_encoder.fit(X[self.one_hot_cols])
            self.one_hot_feature_names = self.one_hot_encoder.get_feature_names_out(self.one_hot_cols)
        
        if self.ordinal_cols:
            self.ordinal_encoder.fit(X[self.ordinal_cols])
        
        return self
    
    def transform(self, X):

        X_transformed = X.copy()
        
        # Apply One-Hot Encoding
        if self.one_hot_cols:
            one_hot_encoded = self.one_hot_encoder.transform(X[self.one_hot_cols])
            one_hot_df = pd.DataFrame(one_hot_encoded, columns=self.one_hot_feature_names, index=X.index)
            X_transformed = pd.concat([X_transformed, one_hot_df], axis=1)
            X_transformed.drop(self.one_hot_cols, axis=1, inplace=True)
        
        # Apply Ordinal Encoding
        if self.ordinal_cols:
            ordinal_encoded = self.ordinal_encoder.transform(X[self.ordinal_cols])
            ordinal_df = pd.DataFrame(ordinal_encoded, columns=self.ordinal_cols, index=X.index)
            X_transformed[self.ordinal_cols] = ordinal_df
        
        return X_transformed
    
    def fit_transform(self, X, y = None):
        return self.fit(X).transform(X)

In [8]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class CorrelationFeatureDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.features_to_drop = []
        self.high_corr_pairs = []
        
    def fit(self, X, y): 
        X_corr = X.copy()
        X_corr['isFraud'] = y
        corr_matrix = X_corr.corr().abs()
        
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                
                if corr_matrix.iloc[i, j] > self.threshold:
                    self.high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
                    
        for feat1, feat2, _ in self.high_corr_pairs:
            if abs(X[feat1].corr(y)) < abs(X[feat2].corr(y)):
                self.features_to_drop.append(feat1)
            else:
                self.features_to_drop.append(feat2)
        
        self.features_to_drop = list(set(self.features_to_drop))
        return self

    
    def transform(self, X):
      return X.drop(columns=self.features_to_drop)
        
    def fit_transform(self, X, y):
        return self.fit(X, y).transform(X)
    

In [9]:
pip uninstall scikit-learn imbalanced-learn -y

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: imbalanced-learn 0.13.0
Uninstalling imbalanced-learn-0.13.0:
  Successfully uninstalled imbalanced-learn-0.13.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.2.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

# Assuming you've already defined your custom classes
pipeline = ImbPipeline([
    ('cleaner', DataCleaner()),
    ('encoder', CustomEncoder()),
    ('correlation_dropper', CorrelationFeatureDropper()),
    ('smote', SMOTE(random_state=42)),  # Handling imbalance
    ('classifier', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))
])

In [12]:
import mlflow
from sklearn.metrics import roc_auc_score

# Set up MLflow experiment
mlflow.set_experiment("Basic_log_regression")

# Start a new run
with mlflow.start_run(run_name="initial_run"):
    # Log parameters
    mlflow.log_params({
        "model_type": "LogisticRegression",
        "imbalance_handling": "SMOTE + class_weight='balanced'",
        "missing_values": "mean for numeric, most_frequent for categorical",
        "encoding": "WOE + one_hot_encoding(columns with unique<3)",
        "feature_selection": "CorrelationFeatureDropper"
    })
    
    # Train and evaluate model
    pipeline.fit(X_train, y_train)
    val_preds = pipeline.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, val_preds)
    
    # Log metrics
    mlflow.log_metric("val_roc_auc", roc_auc)
    print(f"Logged ROC-AUC: {roc_auc:.4f}")
    
    # Log the model
    mlflow.sklearn.log_model(pipeline, "model")
    
    # Add a tag to identify this as baseline
    mlflow.set_tag("stage", "baseline")


Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7eb95c148860>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


Logged ROC-AUC: 0.7453


2025/04/24 21:51:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run initial_run at: https://dagshub.com/zeliz22/ML_House-Pricing.mlflow/#/experiments/7/runs/2e6de227901a4ae0ae548913a1edbf83
🧪 View experiment at: https://dagshub.com/zeliz22/ML_House-Pricing.mlflow/#/experiments/7
